In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [2]:
files = glob.glob("/Nishika/apartments_price/train/original_data/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0, low_memory=False, encoding="utf-8"))
df = pd.concat(data_list)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 637351 entries, 40108844 to 12009989
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   種類            637351 non-null  object 
 1   地域            0 non-null       float64
 2   市区町村コード       637351 non-null  int64  
 3   都道府県名         637351 non-null  object 
 4   市区町村名         637351 non-null  object 
 5   地区名           637060 non-null  object 
 6   最寄駅：名称        634732 non-null  object 
 7   最寄駅：距離（分）     614306 non-null  object 
 8   間取り           615609 non-null  object 
 9   面積（㎡）         637351 non-null  object 
 10  土地の形状         0 non-null       float64
 11  間口            0 non-null       float64
 12  延床面積（㎡）       0 non-null       float64
 13  建築年           619117 non-null  object 
 14  建物の構造         623423 non-null  object 
 15  用途            591214 non-null  object 
 16  今後の利用目的       275091 non-null  object 
 17  前面道路：方位       0 non-null       float64
 18  

In [4]:
df.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
40108844,中古マンション等,NaN,40218,福岡県,春日市,昇町,博多南,23,３ＬＤＫ,70,...,NaN,NaN,NaN,第２種住居地域,60.0,200.0,2012年第１四半期,改装済,NaN,7.230449
40001245,中古マンション等,NaN,40103,福岡県,北九州市若松区,久岐の浜,若松,10,４ＬＤＫ,95,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2015年第４四半期,改装済,NaN,6.875061
40068753,中古マンション等,NaN,40132,福岡県,福岡市博多区,古門戸町,中洲川端,4,２ＬＤＫ,50,...,NaN,NaN,NaN,商業地域,80.0,500.0,2010年第１四半期,未改装,NaN,7.041393
40009332,中古マンション等,NaN,40109,福岡県,北九州市八幡西区,南王子町,黒崎,25,３ＬＤＫ,70,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2018年第１四半期,未改装,NaN,7.041393
40030654,中古マンション等,NaN,40137,福岡県,福岡市早良区,百道浜,西新,16,３ＬＤＫ,120,...,NaN,NaN,NaN,商業地域,80.0,400.0,2019年第２四半期,未改装,NaN,8.000000


## データのクリーニングを行う

In [6]:
def data_pre(df):
    # すべてがNULL値の列を削除
    nonnull_list = [col for col in df.columns if df[col].count() == 0]
    df = df.drop(nonnull_list, axis=1)

    # 不要な列の削除
    if "市区町村名" in df.columns:
        df = df.drop("市区町村名", axis=1)
    if "種類" in df.columns:
        df = df.drop("種類", axis=1)

    # 最寄駅の距離を数値に変換
    dis = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "2H?": 120,
        "1H30?2H": 105,
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    # 面積を数値に変換
    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

    # 建築年の変換
    df["建築年"] = df["建築年"].astype(str)
    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 35 - num
        elif "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        elif "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        elif "戦前" in i:
            year = 76.0
        else:
            year = np.nan  # 欠損値はNaNのまま保持

        y_list[i] = year

    # 建築年を変換してfloat型に
    df["建築年"] = df["建築年"].replace(y_list).astype(float)

    # 取引時点の変換
    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    
    # 取引時点の置換
    for k, j in year.items():
        df["取引時点"] = df["取引時点"].str.replace(k, j, regex=False)

    # 取引時点をfloat型に変換
    df["取引時点"] = df["取引時点"].astype(float)

    # 特定の列をカテゴリ型に変換
    categorical_columns = [
        "都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造",
        "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"
    ]
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df

# 前処理の適用
df_processed = data_pre(df)


/var/folders/ys/5xpvjn3d5p96rvqpz31hvh380000gn/T/ipykernel_1900/4087755602.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["建築年"] = df["建築年"].replace(y_list).astype(float)


In [ ]:
# ディレクトリが存在するか確認し、存在しない場合は作成
save_dir = '/Nishika/apartments_price/train'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 新しい特徴量を作成してデータを保存
save_path = os.path.join(save_dir, 'train_processed.csv')
df_processed.to_csv(save_path, index=False, encoding='utf-8')

print(f"データを保存しました: {save_path}")
